Introduction
============

In most presentations of Riemannian geometry, e.g. @o1983semi and
[Wikipedia](https://en.wikipedia.org/wiki/Fundamental_theorem_of_Riemannian_geometry),
the fundamental theorem of Riemannian geometry ("the miracle of
Riemannian geometry") is given: that for any semi-Riemannian manifold
there is a unique torsion-free metric connection. I assume partly
because of this and partly because the major application of Riemannian
geometry is General Relativity, connections with torsion are given
little if any attention.

It turns out we are all very familiar with a connection with torsion:
the Mercator projection. Some mathematical physics texts,
e.g. @Nakahara2003, allude to this but leave the details to the
reader. Moreover, this connection respects the metric induced from
Euclidean space.

We use [SageManifolds](http://sagemanifolds.obspm.fr) to assist with
the calculations. We hint at how this might be done more slickly in
[Haskell](https://www.haskell.org).

Colophon
--------

This is written as an [Jupyter](http://jupyter.org) notebook. In
theory, it should be possible to run it assuming you have installed at
least sage and Haskell. To publish it, I used

    jupyter-nbconvert --to markdown Mercator.ipynb
    pandoc -s Mercator.md -t markdown+lhs -o Mercator.lhs \
           --filter pandoc-citeproc --bibliography DiffGeom.bib
    BlogLiteratelyD --wplatex Mercator.lhs > Mercator.html

Not brilliant but good enough.

In [200]:
%display latex

In [201]:
viewer3D = 'tachyon'

Warming Up With SageManifolds
=============================

Let us try a simple exercise: finding the connection coefficients of
the Levi-Civita connection for the Euclidean metric on $\mathbb{R}^2$
in polar co-ordinates.

Define the manifold.

In [202]:
N = Manifold(2, 'N',r'\mathcal{N}', start_index=1)

Define a chart and frame with Cartesian co-ordinates.

In [203]:
ChartCartesianN.<x,y> = N.chart()

In [204]:
FrameCartesianN = ChartCartesianN.frame()

Define a chart and frame with polar co-ordinates.

In [205]:
ChartPolarN.<r,theta> = N.chart()

In [206]:
FramePolarN = ChartPolarN.frame()

The standard transformation from Cartesian to polar co-ordinates.

In [207]:
cartesianToPolar = ChartCartesianN.transition_map(ChartPolarN, (sqrt(x^2 + y^2), arctan(y/x)))

In [208]:
print(cartesianToPolar)

Change of coordinates from Chart (N, (x, y)) to Chart (N, (r, theta))


In [209]:
print(cartesianToPolar.display())

r = sqrt(x^2 + y^2)
theta = arctan(y/x)


In [210]:
cartesianToPolar.set_inverse(r * cos(theta), r * sin(theta))

Check of the inverse coordinate transformation:
   x == x
   y == y
   r == abs(r)
   theta == arctan(sin(theta)/cos(theta))


Now we define the metric to make the manifold Euclidean.

In [211]:
g_e = N.metric('g_e')

In [212]:
g_e[1,1], g_e[2,2] = 1, 1

We can display this in Cartesian co-ordinates.

In [213]:
print(g_e.display(FrameCartesianN))


g_e = dx*dx + dy*dy


And we can display it in polar co-ordinates

In [214]:
g_e.display(FramePolarN)

g_e = dr*dr + (x^2 + y^2) dtheta*dtheta

In [215]:
print(g_e.display(FramePolarN))

g_e = dr*dr + (x^2 + y^2) dtheta*dtheta


Next let us compute the Levi-Civita connection from this metric.

In [216]:
nab_e = g_e.connection()


In [217]:
print(nab_e)


Levi-Civita connection nabla_g_e associated with the Riemannian metric g_e on the 2-dimensional differentiable manifold N


If we use Cartesian co-ordinates, we expect that $\Gamma^k_{ij} = 0,
\forall i,j,k$. Only non-zero entries get printed.

In [218]:
print(nab_e.display(FrameCartesianN))

Just to be sure, we can print out all the entries.

In [219]:
print(nab_e[:])

[[[0, 0], [0, 0]], [[0, 0], [0, 0]]]


In polar co-ordinates, we get

In [220]:
print(nab_e.display(FramePolarN))

Gam^r_theta,theta = -sqrt(x^2 + y^2) 
Gam^theta_r,theta = 1/sqrt(x^2 + y^2) 
Gam^theta_theta,r = 1/sqrt(x^2 + y^2) 


In [221]:
print(latex(nab_e.display(FramePolarN)))

\begin{array}{lcl} \Gamma_{ \phantom{\, r } \, \theta \, \theta }^{ \, r \phantom{\, \theta } \phantom{\, \theta } } & = & -\sqrt{x^{2} + y^{2}} \\ \Gamma_{ \phantom{\, \theta } \, r \, \theta }^{ \, \theta \phantom{\, r } \phantom{\, \theta } } & = & \frac{1}{\sqrt{x^{2} + y^{2}}} \\ \Gamma_{ \phantom{\, \theta } \, \theta \, r }^{ \, \theta \phantom{\, \theta } \phantom{\, r } } & = & \frac{1}{\sqrt{x^{2} + y^{2}}} \end{array}


Which we can rew-rewrite as

$$
\begin{aligned}
\Gamma^r_{\theta,\theta} &= -r \\
\Gamma^\theta_{r,\theta} &= 1/r \\
\Gamma^\theta_{\theta,r} &= 1/r
\end{aligned}
$$

with all other entries being 0.

\begin{array}{lcl} \Gamma_{ \phantom{\, r } \, \theta \, \theta }^{ \, r \phantom{\, \theta } \phantom{\, \theta } } & = & -\sqrt{x^{2} + y^{2}} \\ \Gamma_{ \phantom{\, \theta } \, r \, \theta }^{ \, \theta \phantom{\, r } \phantom{\, \theta } } & = & \frac{1}{\sqrt{x^{2} + y^{2}}} \\ \Gamma_{ \phantom{\, \theta } \, \theta \, r }^{ \, \theta \phantom{\, \theta } \phantom{\, r } } & = & \frac{1}{\sqrt{x^{2} + y^{2}}} \end{array}


The Sphere
==========

We define a 2 dimensional manifold.


In [223]:
S2 = Manifold(2, 'S^2', latex_name=r'\mathbb{S}^2', start_index=1)

In [224]:
S2

2-dimensional differentiable manifold S^2

In [225]:
print(S2)

2-dimensional differentiable manifold S^2


In [226]:
# isinstance(S2, Parent)

To start off with we cover the manifold with two charts.

In [229]:
U = S2.open_subset('U')

Open subset U of the 2-dimensional differentiable manifold S^2


In [228]:
V = S2.open_subset('V')

Open subset V of the 2-dimensional differentiable manifold S^2


In [27]:
S2.declare_union(U, V)

In [28]:
stereoN.<x,y> = U.chart()

In [237]:
print(stereoN)

Chart (U, (x, y))


In [31]:
stereoS.<xp,yp> = V.chart(r"xp:x' yp:y'")

In [238]:
print(stereoS)

Chart (V, (xp, yp))


In [0]:
We now have two charts.

In [0]:
S2.atlas()

The charts are the stereographic projections from the North and South
poles but we only need specify transition maps between the charts. See
the example 2.9a in these
[notes](https://www.math.lsu.edu/~lawson/Chapter2.pdf) for more
details.

In [34]:
stereoN_to_S = stereoN.transition_map(stereoS, (x/(x^2+y^2), y/(x^2+y^2)), intersection_name='W',
                                      restrictions1= x^2+y^2!=0, restrictions2= xp^2+xp^2!=0)
stereoN_to_S.display()

xp = x/(x^2 + y^2)
yp = y/(x^2 + y^2)

In [35]:
stereoS_to_N = stereoN_to_S.inverse()
stereoS_to_N.display()

x = xp/(xp^2 + yp^2)
y = yp/(xp^2 + yp^2)

In [36]:
R.<t> = RealLine() ; print(R)

Real number line R


In [37]:
S2.atlas()

[Chart (U, (x, y)),
 Chart (V, (xp, yp)),
 Chart (W, (x, y)),
 Chart (W, (xp, yp))]

In [38]:
W = U.intersection(V)

In [39]:
stereoN_W = stereoN.restrict(W)
stereoN_W

Chart (W, (x, y))

In [40]:
stereoS_W = stereoS.restrict(W)
stereoS_W

Chart (W, (xp, yp))

In [41]:
stereoS_W.plot()

Graphics object consisting of 17 graphics primitives

In [42]:
stereoS_W.plot().save("diagrams/sample.png")

In [43]:
A = W.open_subset('A', coord_def={stereoN_W: (y!=0, x<0), stereoS_W: (yp!=0, xp<0)})
print(A)

Open subset A of the 2-dimensional differentiable manifold S^2


In [44]:
stereoN_A = stereoN_W.restrict(A)
stereoN_A

Chart (A, (x, y))

In [45]:
spher.<th,ph> = A.chart(r'th:(0,pi):\theta ph:(0,2*pi):\phi') ; spher

Chart (A, (th, ph))

In [46]:
loxx.<thhh,phhh> = A.chart(r'thhh:(-oo,oo):\xi phhh:(0,2*pi):\zeta') ; loxx

Chart (A, (thhh, phhh))

In [47]:
spher_to_stereoN = spher.transition_map(stereoN_A, (sin(th)*cos(ph)/(1-cos(th)),
                                                    sin(th)*sin(ph)/(1-cos(th))) )
spher_to_stereoN.display()

x = -cos(ph)*sin(th)/(cos(th) - 1)
y = -sin(ph)*sin(th)/(cos(th) - 1)

In [48]:
spher_to_lox = spher.transition_map(loxx, (log(tan(th/2)), ph))
spher_to_lox.display()

thhh = log(tan(1/2*th))
phhh = ph

In [49]:
spher_to_stereoN.set_inverse(2*atan(1/sqrt(x^2+y^2)), atan2(-y,-x)+pi,
                             check=True)

Check of the inverse coordinate transformation:
   th == 2*arctan(sqrt(-cos(th) + 1)/sqrt(cos(th) + 1))
   ph == pi + arctan2(sin(ph)*sin(th)/(cos(th) - 1), cos(ph)*sin(th)/(cos(th) - 1))
   x == x
   y == y


In [50]:
spher_to_lox.set_inverse(2*arctan(exp(thhh)),phhh,check=True)

Check of the inverse coordinate transformation:
   th == 2*arctan(sin(1/2*th)/cos(1/2*th))
   ph == ph
   thhh == thhh
   phhh == phhh


In [51]:
spher_to_stereoN.inverse().display()

th = 2*arctan(1/sqrt(x^2 + y^2))
ph = pi + arctan2(-y, -x)

In [52]:
spher_to_lox.inverse().display()

th = 2*arctan(e^thhh)
ph = phhh

In [53]:
spher.plot(stereoN, nb_values=15, ranges={th: (pi/8,pi)})

Graphics object consisting of 30 graphics primitives

In [54]:
print(R.category())

Category of smooth manifolds over Real Field with 53 bits of precision


In [55]:
print(dim(R))

1


In [56]:
c = S2.curve({spher: [2*atan(exp(-t)), t]}, (t, -oo, +oo), name='c')

In [57]:
c.parent()

Set of Morphisms from Real number line R to 2-dimensional differentiable manifold S^2 in Category of smooth manifolds over Real Field with 53 bits of precision

In [58]:
c.display()

c: R --> S^2
   t |--> (x, y) = (cos(t)*e^t, e^t*sin(t))
   t |--> (xp, yp) = (cos(t)*e^(-t), e^(-t)*sin(t))
   t |--> (th, ph) = (2*arctan(e^(-t)), t)
   t |--> (thhh, phhh) = (-t, t)

In [59]:
c.plot(chart=stereoN, aspect_ratio=1)

Graphics object consisting of 1 graphics primitive

In [60]:
c.plot(chart=spher, aspect_ratio=0.2)

Graphics object consisting of 1 graphics primitive

In [61]:
c.plot(chart=loxx, aspect_ratio=0.5)

Graphics object consisting of 1 graphics primitive

In [62]:
# A.default_chart()

In [63]:
# A.default_frame()

In [64]:
# A.set_default_chart(spher)
# A.set_default_frame(spher.frame())
# A.default_chart()

In [65]:
# A.default_frame()

In [66]:
# a = A.automorphism_field()
# a[1,1], a[2,2] = 1, 1/sin(x)
# a.display()

In [67]:
# a[:]

In [68]:
# e = spher.frame().new_frame(a, 'e')
# print(e) ; e

In [69]:
# e[1].display()

In [70]:
# e[2].display()

In [71]:
# A.frames()

In [85]:
print(A.default_frame())

Coordinate frame (A, (d/dx,d/dy))


In [87]:
A.set_default_chart(spher)
A.set_default_frame(spher.frame())
print(A.default_chart())

Chart (A, (th, ph))


In [89]:
print(A.default_frame())

Coordinate frame (A, (d/dth,d/dph))


In [91]:
a = A.automorphism_field()
a[1,1], a[2,2] = 1, 1/sin(th)
print(a.display())

d/dth*dth + 1/sin(th) d/dph*dph


In [92]:
print(a[:])

[        1         0]
[        0 1/sin(th)]


In [93]:
e = spher.frame().new_frame(a, 'e')
print(e) ; e

Vector frame (A, (e_1,e_2))

Vector frame (A, (e_1,e_2))


In [95]:
print(e[1].display())

e_1 = d/dth


In [96]:
print(e[2].display())

e_2 = 1/sin(th) d/dph


In [97]:
A.frames()

[Coordinate frame (A, (d/dx,d/dy)),
 Coordinate frame (A, (d/dxp,d/dyp)),
 Coordinate frame (A, (d/dth,d/dph)),
 Coordinate frame (A, (d/dthhh,d/dphhh)),
 Vector frame (A, (e_1,e_2))]

In [134]:
f2 = A.frames()[2]

In [135]:
print(f2)

Coordinate frame (A, (d/dth,d/dph))


In [144]:
f3 = A.frames()[3]

In [145]:
print(f3)

Coordinate frame (A, (d/dthhh,d/dphhh))


In [136]:
print(e.structure_coef()[:])

[[[0, 0], [0, 0]], [[0, -cos(th)/sin(th)], [cos(th)/sin(th), 0]]]


In [137]:
nab_l = A.affine_connection('nabla_l', latex_name=r'\nabla_l')

In [138]:
nab_l.add_coef(e)

3-indices components w.r.t. Vector frame (A, (e_1,e_2))

In [139]:
print(nab_l)

Affine connection nabla_l on the Open subset A of the 2-dimensional differentiable manifold S^2


In [140]:
nab_l.display(e)

In [141]:
print(nab_l.coef()[:])

[[[0, 0], [0, 0]], [[0, 0], [cos(th)/sin(th), 0]]]


In [142]:
print(nab_l.coef(f2)[:])

[[[0, 0], [0, 0]], [[0, 0], [cos(th)/sin(th), 0]]]


In [146]:
print(nab_l.coef(f3)[:])

[[[2*cos(1/2*th)^2 - 1, 0], [0, 0]], [[0, 0], [2*cos(1/2*th)*cos(th)*sin(1/2*th)/sin(th), 0]]]


In [150]:
nab_l.display(f3)

Gam^thhh_thhh,thhh = 2*cos(1/2*th)^2 - 1 
Gam^phhh_phhh,thhh = 2*cos(1/2*th)*cos(th)*sin(1/2*th)/sin(th) 

$2\cos(\theta/2)^2 - 1 = \cos\theta$ and $2\cos\theta/2\sin\theta/2 =
\sin\theta$.

In [143]:
print(nab_l.coef(e)[:])

[[[0, 0], [0, 0]], [[0, 0], [0, 0]]]


In [125]:
print(nab_l.torsion().display())

cos(th)/sin(th) d/dph*dth*dph - cos(th)/sin(th) d/dph*dph*dth


In [157]:
M = Manifold(2, 'M',r'\mathcal{M}', start_index=1)

In [158]:
X.<x,y> = M.chart()

In [159]:
# X.add_restrictions([x > -pi/2 , x < pi/2, y > 0, y < 2*pi])

In [160]:
Y.<u,v> = M.chart()


In [161]:
# Y.add_restrictions([v > 0, v < 2*pi])

In [162]:
# Z.<w,z> = M.chart()

In [163]:
xy_to_uv = X.transition_map(Y, (log(tan(x/2)), y))

In [164]:
# xy_to_wz = X.transition_map(Z, (2*arctan(exp(x)), y))

In [165]:
print(xy_to_uv)

Change of coordinates from Chart (M, (x, y)) to Chart (M, (u, v))


In [166]:
# print(xy_to_wz)

In [167]:
xy_to_uv.display()

u = log(tan(1/2*x))
v = y

In [168]:
xy_to_uv.set_inverse(2*arctan(exp(u)), v)

Check of the inverse coordinate transformation:
   x == 2*arctan(sin(1/2*x)/cos(1/2*x))
   y == y
   u == u
   v == v


In [169]:
# xy_to_wz.display()

In [170]:
# xy_to_wz.set_inverse(log(tan(w/2)), z)

In [171]:
xy_to_uv.inverse().display()

x = 2*arctan(e^u)
y = v

In [172]:
# xy_to_wz.inverse().display()

In [173]:
print(X)

Chart (M, (x, y))


In [174]:
print(Y)

Chart (M, (u, v))


In [175]:
# print(Z)

In [176]:
eX = X.frame()

In [177]:
eY = Y.frame()

In [178]:
# eZ = Z.frame()

Add the metric induced from Euclidean metric in $\mathbb{R}^3$.

In [179]:
g = M.metric('g')

In [180]:
g[1,1], g[2,2] = 1, (sin(x))^2

In [181]:
print(g.display(eX))

g = dx*dx + sin(x)^2 dy*dy


As expected by the "miracle" of semi-Riemann geometry this gives a
torsion-free connection which respects the metric.



In [182]:
nab_g = g.connection()

In [183]:
print(nab_g)

Levi-Civita connection nabla_g associated with the Riemannian metric g on the 2-dimensional differentiable manifold M


In [184]:
print(nab_g.display(eX))

Gam^x_yy = -cos(x)*sin(x) 
Gam^y_xy = cos(x)/sin(x) 
Gam^y_yx = cos(x)/sin(x) 


In [185]:
print(nab_g.torsion().display(eX))

0


In [186]:
nab_g(g).display()

nabla_g(g) = 0

Create a connection with as yet no connection co-efficients.

In [187]:
nab = M.affine_connection('nabla', latex_name=r'\nabla')


In [296]:
ch_basis = M.automorphism_field()

In [297]:
ch_basis[1,1], ch_basis[2,2] = 1, sin(x)

In [298]:
e = M.default_frame().new_frame(ch_basis, 'e')

In [649]:
nab.add_coef(e)


3-indices components w.r.t. Vector frame (M, (e_1,e_2))

In [650]:
print(nab.display(e))

In [651]:
print(nab.display(eX))

Gam^y_yx = -cos(x)/sin(x) 


In [652]:
nab._coefficients

{Coordinate frame (M, (d/dx,d/dy)): 3-indices components w.r.t. Coordinate frame (M, (d/dx,d/dy)),
 Vector frame (M, (e_1,e_2)): 3-indices components w.r.t. Vector frame (M, (e_1,e_2))}

In [653]:
nab.coef()[:]

[[[0, 0], [0, 0]], [[0, 0], [-cos(x)/sin(x), 0]]]

In [654]:
nab.coef(eY)[:]

[[[2*cos(1/2*x)^2 - 1, 0], [0, 0]],
 [[0, 0], [-2*cos(1/2*x)*cos(x)*sin(1/2*x)/sin(x), 0]]]

$$
\begin{align}
{\ddot{\gamma}}^x \phantom{ + \frac{\cos x}{\sin x}\dot{\gamma}^y\dot{\gamma}^x} & = 0 \\
{\ddot{\gamma}}^y - \frac{\cos x}{\sin x}\dot{\gamma}^y\dot{\gamma}^x & = 0
\end{align}
$$

For now let us assume that the solution to ${\ddot{\gamma}}^x = 0$ is
$\gamma^x = t$. Then we can see that a solution to the second equation is
$\gamma^y = \log\tan(t/2)$.


In [655]:
print(nab.display(eY))

Gam^u_uu = 2*cos(1/2*x)^2 - 1 
Gam^v_vu = -2*cos(1/2*x)*cos(x)*sin(1/2*x)/sin(x) 


In [656]:
# print(nab.display(eZ))

$2\cos^2{x/2} - 1 = \cos{x}$ and $2\sin{x/2}\cos{x/2} = \sin{x}$ so
that $\Gamma^u_{uu} = \cos{x}$ and $\Gamma^v_{vu} = -\cos{x}$.

$$
\begin{align}
{\ddot{\gamma}}^u + \cos x\dot{\gamma}^u\dot{\gamma}^u & = 0 \\
{\ddot{\gamma}}^v - \cos x\dot{\gamma}^v\dot{\gamma}^u & = 0
\end{align}
$$


In [657]:
print(nab.torsion().display())

-cos(x)/sin(x) d/dy*dx*dy + cos(x)/sin(x) d/dy*dy*dx


In [658]:
nab(g).display()

nabla(g) = 4*cos(x)*sin(x) dy*dy*dx

The **Koszul formula** (see e.g. [@o1983semi]) characterizes the
Levi-Civita connection $\nabla$

$$
\begin{align}
2  \langle \nabla_X Y, Z\rangle & = X  \langle Y,Z\rangle + Y  \langle Z,X\rangle - Z  \langle X,Y\rangle \\
&-  \langle X,[Y,Z]\rangle +   \langle Y,[Z,X]\rangle +  \langle Z,[X,Y]\rangle
\end{align}
$$

Being more explicit about the metric, this can be re-written as

$$
\begin{align}
2 g(\nabla^g_X Y, Z) & = X g(Y,Z) + Y g(Z,X) - Z g(X,Y) \\
&- g(X,[Y,Z]) +  g(Y,[Z,X]) + g(Z,[X,Y])
\end{align}
$$


Let $\nabla^\tilde{g}$ be the Levi-Civita connection for the metric
$\tilde{g} = e^{2\sigma}g$ where $\sigma \in C^\infty M$. Following
[Gadea2010] and substituting into the Koszul formula and then applying
the product rule

$$
\begin{align}
2 e^{2 \sigma} g(\nabla^\tilde{g}_X Y, Z) & = X  e^{2 \sigma} g(Y,Z) + Y e^{2 \sigma} g(Z,X) - Z e^{2 \sigma} g(X,Y) \\
& + e^{2 \sigma} g([X,Y],Z]) - e^{2 \sigma} g([Y,Z],X) + e^{2 \sigma} g([Z,X],Y) \\
& = 2 e^{2\sigma}[g(\nabla^{g}_X Y, Z) + X\sigma g(Y,Z) + Y\sigma g(Z,X) - Z\sigma g(X,Y)] \\
& = 2 e^{2\sigma}[g(\nabla^{g}_X Y + X\sigma Y + Y\sigma X - g(X,Y) \operatorname{grad}\sigma, Z)]
\end{align}
$$


Where as usual the vector field, $\operatorname{grad}\phi$ for $\phi
\in C^\infty M$, is defined via $g(\operatorname{grad}\phi, X) =
\mathrm{d}\phi(X) = X\phi$.

In [659]:
h = M.metric('h')

In [660]:
h[1,1], h[2,2] = 1 / (sin(x))^2, 1

In [661]:
print(h.display(eX))


h = sin(x)^(-2) dx*dx + dy*dy


In [662]:
nab_h = h.connection()

In [663]:
print(nab_h)

Levi-Civita connection nabla_h associated with the Riemannian metric h on the 2-dimensional differentiable manifold M


In [664]:
print(nab_h.display(eX))

Gam^x_xx = -cos(x)/sin(x) 


In [665]:
v = M.vector_field('v')

In [666]:
v[eX,:] = [cos(x)/sin(x), 0]

In [667]:
print(v.display(eX))

v = cos(x)/sin(x) d/dx


In [668]:
print(v.display(eY))

v = 1/2*cos(x)/(cos(1/2*x)*sin(1/2*x)*sin(x)) d/du


In [669]:
print(nab_g.display())

Gam^x_yy = -cos(x)*sin(x) 
Gam^y_xy = cos(x)/sin(x) 
Gam^y_yx = cos(x)/sin(x) 


In [670]:
e_x = M.vector_field('e_x'); e_y = M.vector_field('e_y')

In [671]:
e_x[:] = [1, 0]; e_y[:] = [0, 1]

In [672]:
print(nab_g(e_x).display())

nabla_g(e_x) = cos(x)/sin(x) d/dy*dy


In [673]:
print(nab_g(e_y).display())

nabla_g(e_y) = -cos(x)*sin(x) d/dx*dy + cos(x)/sin(x) d/dy*dx


$\nabla_{\partial_i}{\partial_j} = \Gamma^k_{ij}\partial_k$

In [674]:
print(nab_g(e_y)[:])

[             0 -cos(x)*sin(x)]
[ cos(x)/sin(x)              0]


In [675]:
print(nab_g(e_x)(e_x).display())

nabla_g(e_x)(e_x) = 0


In [676]:
print(nab_g(e_y)(e_x).display())


nabla_g(e_y)(e_x) = cos(x)/sin(x) d/dy


In [677]:
print(nab_g(e_x)(e_y).display())

nabla_g(e_x)(e_y) = cos(x)/sin(x) d/dy


In [678]:
print(nab_g(e_y)(e_y).display())

nabla_g(e_y)(e_y) = -cos(x)*sin(x) d/dx


In [679]:
print(v.display())

v = cos(x)/sin(x) d/dx


In [680]:
f = M.scalar_field(-ln(sin(x)), name='f')

In [681]:
f.display()

f: M --> R
   (x, y) |--> -log(sin(x))
   (u, v) |--> -u + log(1/2*e^(2*u) + 1/2)

In [682]:
e_x(f)

Scalar field e_x(f) on the 2-dimensional differentiable manifold M

In [683]:
e_x(f).display()

e_x(f): M --> R
   (x, y) |--> -cos(x)/sin(x)
   (u, v) |--> 1/2*(e^(2*u) - 1)*e^(-u)

In [684]:
nab_g(e_x)(e_x).display()

nabla_g(e_x)(e_x) = 0

In [685]:
(e_x(f) * e_x).display()

-cos(x)/sin(x) d/dx

In [686]:
g(e_x,e_x).display()

g(e_x,e_x): M --> R
   (x, y) |--> 1
   (u, v) |--> 1

In [687]:
nab_tilde_xx = nab_g(e_x)(e_x) + e_x(f) * e_x + e_x(f) * e_x + g(e_x,e_x) * e_x * cos(x) / sin(x)

In [688]:
nab_tilde_xx.display()

-cos(x)/sin(x) d/dx

In [689]:
nab_tilde_xy = nab_g(e_y)(e_x) + e_x(f) * e_y + e_y(f) * e_x + g(e_x,e_y) * e_x * cos(x) / sin(x)

In [690]:
nab_tilde_xy.display()

0

In [691]:
nab_tilde_yx = nab_g(e_x)(e_y) + e_y(f) * e_x + e_x(f) * e_y + g(e_y,e_x) * e_x * cos(x) / sin(x)

In [692]:
nab_tilde_yx.display()

0

In [693]:
nab_tilde_yy = nab_g(e_y)(e_y) + e_y(f) * e_y + e_y(f) * e_y + g(e_y,e_y) * e_x * cos(x) / sin(x)

In [694]:
nab_tilde_yy.display()

0

In [695]:
nab_tilde = M.affine_connection('nabla_t', r'\tilde_{\nabla}')

In [696]:
print(nab_tilde)

Affine connection nabla_t on the 2-dimensional differentiable manifold M


In [697]:
for i in M.irange():
    for j in M.irange():
        for k in M.irange():
            nab_tilde.add_coef()[k,i,j] = \
                nab_g(X.frame()[i])(X.frame()[j])(X.coframe()[k]) + \
                X.frame()[i](f) * X.frame()[j](X.coframe()[k]) + \
                X.frame()[j](f) * X.frame()[i](X.coframe()[k]) + \
                g(X.frame()[i], X.frame()[j]) * X.frame()[1](X.coframe()[k]) * cos(x) / sin(x)

In [698]:
nab_tilde.display()

Gam^x_xx = -cos(x)/sin(x) 

In [699]:
print(nab_tilde.display())

Gam^x_xx = -cos(x)/sin(x) 


In [700]:
nab_g.display()

Gam^x_yy = -cos(x)*sin(x) 
Gam^y_xy = cos(x)/sin(x) 
Gam^y_yx = cos(x)/sin(x) 

In [701]:
print(g(X.frame()[1],X.frame()[1]))

Scalar field g(d/dx,d/dx) on the 2-dimensional differentiable manifold M


In [702]:
print(x)

x


In [703]:
print(nab_g(X.frame()[1])(X.frame()[1])(X.coframe()[1]))

Scalar field dx(nabla_g(d/dx)(d/dx)) on the 2-dimensional differentiable manifold M


In [704]:
foo = nab_g(X.frame()[1])(X.frame()[1])(X.coframe()[1])

In [705]:
foo.display()

dx(nabla_g(d/dx)(d/dx)): M --> R
   (x, y) |--> 0
   (u, v) |--> 0

In [706]:
dx = M.default_frame().coframe()

In [707]:
check = []

In [708]:
for i in M.irange():
    for j in M.irange():
            check.append( nab.connection_form(i,j) == \
                          sum( nab[[i,j,k]]*dx[k] for k in M.irange() ) )

In [709]:
check

[True, True, True, True]

In [157]:
print(latex(x^2))

x^{2}


Further Reading
===============
More details can be found in:

 * @Agricola2004

References
==========